<a href="https://colab.research.google.com/github/LowForest/face_detector/blob/main/examples/face_landmarker/python/%5BMediaPipe_Python_Tasks%5D_Face_Landmarker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

# Face Landmarks Detection with MediaPipe Tasks

## Preparation

Let's start with installing MediaPipe.

In [3]:
!python -m pip uninstall -y -q dopamine-rl albumentations albucore ydf grpcio-status spacy thinc opencv-python opencv-contrib-python opencv-python-headless || true
!pip install -U --quiet pip setuptools wheel
!pip install --quiet "numpy==1.26.4" "protobuf==4.25.3" "opencv-python==4.8.1.78" "mediapipe==0.10.21" "matplotlib>=3.7,<3.9" "jedi>=0.16"

Then download the off-the-shelf model bundle(s). Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker#models) for more information about these model bundles.

In [4]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

## Visualization utilities

In [5]:
import matplotlib.pyplot as plt
import mediapipe as mp
import os, time, base64, types, sys, cv2, numpy as np
from mediapipe import solutions
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
from IPython.display import display, Image as IPyImage


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

## Running inference and visualizing the results

Here are the steps to run face landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python) to learn more about configuration options that this task supports.


In [6]:
# --- 0) JS: khởi tạo webcam + hàm lấy frame, tất cả trong window._mp
eval_js(r"""
(() => {
  if (!window._mp) window._mp = {};
  window._mp.initCam = async () => {
    if (window._mp.stream && window._mp.stream.active) return true;

    // Khối UI riêng cho webcam
    const wrap = document.createElement('div'); wrap.id = '_mp_wrap';
    const video = document.createElement('video'); video.style.maxWidth = '100%';
    const stop  = document.createElement('button'); stop.textContent = '⏹ Stop'; stop.style.margin = '6px 0';
    wrap.appendChild(video); wrap.appendChild(document.createElement('br')); wrap.appendChild(stop);
    document.body.appendChild(wrap);

    const stream = await navigator.mediaDevices.getUserMedia({
      video: {width: {ideal: 960}, height: {ideal: 540}}, audio: false
    });
    video.srcObject = stream; await video.play();

    const canvas = document.createElement('canvas');
    const ctx = canvas.getContext('2d');

    stop.onclick = () => { stream.getTracks().forEach(t => t.stop()); };

    window._mp.wrap = wrap;
    window._mp.video = video;
    window._mp.stream = stream;
    window._mp.canvas = canvas;
    window._mp.ctx = ctx;
    return true;
  };

  window._mp.camActive = () => !!(window._mp && window._mp.stream && window._mp.stream.active);

  window._mp.captureFrame = (q=0.85) => {
    if (!window._mp.camActive()) return null;
    const {video, canvas, ctx} = window._mp;
    if (!video.videoWidth || !video.videoHeight) return null;
    canvas.width = video.videoWidth; canvas.height = video.videoHeight;
    ctx.drawImage(video, 0, 0);
    return canvas.toDataURL('image/jpeg', q); // dataURL JPEG
  };

  window._mp.stopCam = () => {
    if (window._mp && window._mp.stream) window._mp.stream.getTracks().forEach(t=>t.stop());
    return true;
  };
  return true;
})()
""")

# --- 1) Bật webcam & xin quyền
_ = eval_js("window._mp.initCam()")

# --- 2) Chọn model .task đã tải
MODEL_CANDIDATES = ["face_landmarker_v2_with_blendshapes.task", "face_landmarker.task"]
MODEL_PATH = next((p for p in MODEL_CANDIDATES if os.path.exists(p)), None)
assert MODEL_PATH, "Không thấy file model .task (ví dụ: face_landmarker_v2_with_blendshapes.task)."

# --- 3) Tạo Face Landmarker ở VIDEO mode (giữ blendshapes/transform như bạn dùng)
base_vid = python.BaseOptions(model_asset_path=MODEL_PATH)
options_vid = vision.FaceLandmarkerOptions(
    base_options=base_vid,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    min_face_detection_confidence=0.5,
    min_face_presence_confidence=0.5,
    min_tracking_confidence=0.5,
)
detector_stream = vision.FaceLandmarker.create_from_options(options_vid)

# --- 4) Tùy chọn ghi video ra file
RECORD = True
OUTPUT_FILE = "face_stream.mp4"
FOURCC = cv2.VideoWriter_fourcc(*'mp4v')
TARGET_FPS = 15.0
writer = None

# --- 5) Tạo DisplayHandle để cập nhật ảnh đã annotate mà không clear_output
disp = display(IPyImage(data=b"", format='png'), display_id=True)

# --- 6) Loop cho đến khi bạn bấm ⏹ Stop
t_prev = time.perf_counter()
ts_ms = 0
log_prev = 0.0
mat_prev = 0.0

try:
  while eval_js("(window._mp && window._mp.camActive) ? window._mp.camActive() : false"):
    data = eval_js("(window._mp && window._mp.captureFrame) ? window._mp.captureFrame(0.8) : null")
    if not data:
      time.sleep(0.01)
      continue

    # dataURL -> np.uint8 -> BGR
    b64 = data.split(",")[1]
    buf = np.frombuffer(base64.b64decode(b64), dtype=np.uint8)
    frame_bgr = cv2.imdecode(buf, cv2.IMREAD_COLOR)
    if frame_bgr is None:
      continue

    # BGR -> RGB (SRGB) cho MediaPipe
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    mp_img = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # Timestamp tăng dần (bắt buộc VIDEO mode)
    now = time.perf_counter()
    dt_ms = max(1, int((now - t_prev) * 1000))
    ts_ms += dt_ms
    t_prev = now

    # Detect + vẽ bằng hàm của bạn (mesh/contours/iris)
    result = detector_stream.detect_for_video(mp_img, ts_ms)
    # --- ĐẾM SỐ ĐIỂM / SỐ MẶT ---
    faces = result.face_landmarks or []
    num_faces = len(faces)
    pts_each = [len(lmks) for lmks in faces]             # thường ~478 điểm/face
    total_pts = sum(pts_each)

    # Ghi overlay lên khung hình (khi đã có annotated_rgb / annotated_bgr)
    # (giữ nguyên style vẽ của bạn dùng draw_landmarks_on_image)
    annotated_rgb = draw_landmarks_on_image(frame_rgb, result)
    annotated_bgr = cv2.cvtColor(annotated_rgb, cv2.COLOR_RGB2BGR)

    label = f"faces: {num_faces} | pts(each): {pts_each} | total: {total_pts}"
    cv2.putText(annotated_bgr, label, (10, 28),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Throttle in ra console mỗi ~1s để tránh spam
    now_s = time.perf_counter()
    if now_s - log_prev > 1.0:
        print(label)
        log_prev = now_s

    annotated_rgb = draw_landmarks_on_image(frame_rgb, result)   # <-- giữ nguyên style của bạn
    annotated_bgr = cv2.cvtColor(annotated_rgb, cv2.COLOR_RGB2BGR)

    # Ghi file nếu bật RECORD
    if RECORD:
      if writer is None:
        h, w = annotated_bgr.shape[:2]
        writer = cv2.VideoWriter(OUTPUT_FILE, FOURCC, TARGET_FPS, (w, h))
      writer.write(annotated_bgr)

    # Cập nhật preview (không clear_output)
    _, enc = cv2.imencode('.jpg', annotated_bgr)
    disp.update(IPyImage(data=enc.tobytes()))
finally:
  if writer is not None:
    writer.release()
  _ = eval_js("(window._mp && window._mp.stopCam) ? (window._mp.stopCam(), true) : true")
  print(f"Stopped. Video saved to: {os.path.abspath(OUTPUT_FILE) if RECORD else '(recording disabled)'}")

  # --- IN RA FACIAL TRANSFORMATION MATRIXES ---
  # result.facial_transformation_matrixes: danh sách ma trận (1 cái mỗi mặt)
  mats = getattr(result, "facial_transformation_matrixes", None) or []
  now_s = time.perf_counter()

  if mats:
      # In 1 ma trận đầu tiên (mặt 0); reshape về (4,4) nếu cần
      for i, Mi in enumerate(mats):
        Mi = np.array(Mi)
      if Mi.size == 16: Mi = Mi.reshape(4, 4)
      print(f"[face {i}] matrix:\n{Mi}")

      # In ra console ~mỗi 1s để tránh spam
      if now_s - mat_prev > 1.0:
          np.set_printoptions(precision=4, suppress=True)
          print("facial_transformation_matrixes[0]:\n", M0)
          mat_prev = now_s
  else:
      # Không có mặt nào → có thể log thưa hơn nếu muốn
      if now_s - log_prev > 1.0:
          print("No face / no transformation matrix.")
          log_prev = now_s


MessageError: NotFoundError: Requested device not found

We will also visualize the face blendshapes categories using a bar graph.

In [ ]:
plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])

And print the transformation matrix.

In [ ]:
print(detection_result.facial_transformation_matrixes)